In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0-rc1'

### 加载数据 (fashion-minit)

In [3]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [4]:
path = "../../data/fashion_mnist"

In [5]:
text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']

In [6]:
x_train,y_train = load_mnist(path)
x_test,y_test = load_mnist(path,"t10k")

In [7]:
x_train = tf.cast(x_train,tf.float32) / 255
tf.cast(y_train,tf.int32)
x_test = tf.cast(x_test,tf.float32) / 255
tf.cast(y_test,tf.int32)

<tf.Tensor: id=11, shape=(10000,), dtype=int32, numpy=array([9, 2, 1, ..., 8, 1, 5])>

In [8]:
x_train[0],text_labels[y_train[0]]

(<tf.Tensor: id=15, shape=(784,), dtype=float32, numpy=
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.  

### 生成dataset

In [9]:
ds_train = tf.data.Dataset.from_tensor_slices((x_train,y_train))
ds_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))

In [10]:
# @tf.function
# def map_fun(x,y):
#     x = tf.cast(x,tf.float32)
#     return x / 255.0 ,tf.cast(y,tf.int32)

In [11]:
def transformDs(ds):
    return ds.shuffle(5000).batch(32).repeat()

In [12]:
ds_train = transformDs(ds_train)
ds_test = transformDs(ds_test)

In [13]:
next(iter(ds_train))

(<tf.Tensor: id=48, shape=(32, 784), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: id=49, shape=(32,), dtype=uint8, numpy=
 array([4, 7, 2, 5, 6, 3, 5, 6, 1, 9, 9, 1, 7, 0, 2, 4, 6, 3, 3, 9, 5, 2,
        1, 8, 8, 8, 0, 2, 5, 1, 5, 7], dtype=uint8)>)

### 创建模型

In [14]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512,activation="relu"))#,kernel_initializer = tf.keras.initializers.he_normal()
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10,activation="softmax"))#,kernel_initializer = tf.keras.initializers.he_normal()

In [15]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,optimizer=tf.keras.optimizers.SGD(0.03),metrics=["acc"])

In [16]:
import os
import datetime

In [17]:
logdir = os.path.join("..\..\logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir,write_images = True)

In [18]:
steps_per_epoch = len(x_train) // 32
validation_steps = len(x_test) // 32
model.fit(ds_train,epochs=10,steps_per_epoch=steps_per_epoch,validation_data = ds_test,validation_steps=validation_steps,callbacks=[tensorboard_callback])

Train for 1875 steps, validate for 312 steps
Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.6061 - acc: 0.7938 - val_loss: 0.4914 - val_acc: 0.8253
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4476 - acc: 0.8430 - val_loss: 0.4325 - val_acc: 0.8450
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4038 - acc: 0.8577 - val_loss: 0.4272 - val_acc: 0.8463
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3786 - acc: 0.8655 - val_loss: 0.4095 - val_acc: 0.8523
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3601 - acc: 0.8708 - val_loss: 0.3901 - val_acc: 0.8611
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3439 - acc: 0.8771 - val_loss: 0.3825 - val_acc: 0.8628
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3320 - acc: 0.8808 - val_loss: 0.3858 - val_acc: 0.8568
Epoch 8/10
1875/1

In [19]:
from tensorboard import notebook
notebook.list()

Known TensorBoard instances:
  - port 6006: logdir F:\workspace\tensorflow\d2l-zh-tensorflow\logs\ (started 0:00:08 ago; pid 10308)
  - port 6006: logdir F:workspacetensorflowd2l-zh-tensorflowlogs (started 0:15:36 ago; pid 8068)
